In [1]:
import requests
import re
from bs4 import BeautifulSoup
import json
import datetime
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
import pymysql

C:\Users\Paul\miniconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [22]:
URL_MAIN = 'https://www.tickpick.com/concerts/yeat-tickets/'
proxies = {
    'http': 'http://107.151.182.247:80'
}

r = requests.get(URL_MAIN, proxies = proxies, headers={'User-Agent': 'Chrome'})
soup = BeautifulSoup(r.content, 'html.parser')
print(soup)

<!DOCTYPE html>

<html lang="en" xmlns:fb="http://ogp.me/ns/fb#">
<head>
<meta charset="utf-8"/>
<title>Yeat Tickets - No Service Fees</title>
<meta content="Get 100% guaranteed Yeat tickets on TickPick and save 10%. We guarantee the best prices on all of our tickets because we don't charge any service fees." id="MetaDescription" name="DESCRIPTION"/>
<meta content="width=device-width, initial-scale=1, user-scalable=no" name="viewport"/>
<meta content="	©2022 TickPick, LLC. All rights reserved" id="MetaCopyright" name="COPYRIGHT"/>
<meta content="TickPick" id="MetaAuthor" name="AUTHOR"/>
<meta content="INDEX, FOLLOW" id="MetaRobots" name="ROBOTS"/>
<meta content="GENERAL" name="RATING"/>
<meta content="RevealTrans(Duration=0,Transition=1)" http-equiv="PAGE-ENTER"/>
<meta content="chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="DOCUMENT" name="RESOURCE-TYPE"/>
<meta content="autoRotate:disabled" http-equiv="ScreenOrientation"/>
<link href="/favicon.ico" rel="icon" type="image/x-i

In [15]:
urls = []
venue_names = []
dates = []

for script in soup.find_all("script", type='application/ld+json'):
    #print(script.string)
    if '"@type": "Event"' in script.string:
        url = re.findall('"url": "([^\s"]*)"', script.string)[0]
        urls.append(url)

        venue_name = re.findall('"@type": "EventVenue",[\s]*"name": "([^"]*)"*', script.string)[0]
        venue_names.append(venue_name)
        
        date = re.findall('"startDate": "([^T]*)T', script.string)[0]
        dates.append(date)

print(len(dates))
print(len(urls))
print(len(venue_names))

15
15
15


In [16]:
API_URL = 'https://api.tickpick.com/1.0/listings/internal/event/'

date = datetime.datetime.now()
time = date.strftime("%b %d %I:%M%p")

def scrap_page(url, date, venue_name, time):
    r = requests.get(url, proxies = proxies)
    soup = BeautifulSoup(r.content, 'html.parser')
    id = re.findall('https://www.tickpick.com/[^/]+/([^/]+)/', url)[0]
    r = requests.get(API_URL + id)
    soup = BeautifulSoup(r.content, 'html.parser')
    content = json.loads(soup.prettify())
    for ticket in content:
        dic = {}
        dic['Date'] = time
        dic['VenueID'] = venue_name
        dic['DateEvent'] = date
        dic['Quantity'] = ticket['sp'][0]
        dic['Price'] = int(ticket['p'])
        row_list.append(dic)

row_list = []

for i in range(len(dates)):
    print(urls[i])
    scrap_page(urls[i], dates[i], venue_names[i], time)

df = pd.DataFrame(row_list)
df.head()

server = 'tickpick-scraper-database.mysql.database.azure.com'
database = 'tickpick_database'
username = 'pauldoucet'
driver = '{ODBC Driver 17 for SQL Server}'
ca_path = "DigiCertGlobalRootCA.crt.pem"

f = open("password.json")
data = json.load(f)
password = data["password"]

sqlUrl = URL(
    drivername="mysql+pymysql",
    username=username,
    password=password,
    host=server,
    port=3306,
    database=database,
    query={"ssl_ca": ca_path},
)
engine = create_engine(sqlUrl)

df.to_sql("test_table", con=engine, if_exists="append", index=False)

https://www.tickpick.com/buy-yeat-tickets-the-fillmore-silver-spring-4-12-22-8pm/5076024/
https://www.tickpick.com/buy-yeat-tickets-brooklyn-bowl-philadelphia-4-13-22-8pm/5076025/
https://www.tickpick.com/buy-yeat-tickets-the-summit-music-hall-4-15-22-8pm/5076844/
https://www.tickpick.com/buy-yeat-tickets-brighton-music-hall-ma-4-17-22-7pm/5076023/
https://www.tickpick.com/buy-yeat-tickets-irving-plaza-4-18-22-7pm/5076021/
https://www.tickpick.com/buy-yeat-tickets-warsaw-4-19-22-7pm/5076022/
https://www.tickpick.com/buy-yeat-tickets-heaven-stage-at-masquerade-ga-4-21-22-6pm/5076845/
https://www.tickpick.com/buy-yeat-tickets-the-underground-charlotte-4-24-22-8pm/5076276/
https://www.tickpick.com/buy-yeat-tickets-bronze-peacock-at-the-house-of-blues-houston-4-27-22-8pm/5076240/
https://www.tickpick.com/buy-yeat-tickets-the-echo-lounge-and-music-hall-4-28-22-8pm/5076241/
https://www.tickpick.com/buy-yeat-tickets-the-independent-5-8-22-8pm/5076860/
https://www.tickpick.com/buy-yeat-tickets

In [17]:
df.head(20)

,Date,VenueID,DateEvent,Quantity,Price
0,Apr 12 02:27PM,The Fillmore Silver Spring,2022-04-12,2,147
1,Apr 12 02:27PM,The Fillmore Silver Spring,2022-04-12,1,147
2,Apr 12 02:27PM,The Fillmore Silver Spring,2022-04-12,6,157
3,Apr 12 02:27PM,The Fillmore Silver Spring,2022-04-12,1,160
4,Apr 12 02:27PM,The Fillmore Silver Spring,2022-04-12,2,164
5,Apr 12 02:27PM,The Fillmore Silver Spring,2022-04-12,4,176
6,Apr 12 02:27PM,The Fillmore Silver Spring,2022-04-12,1,198
7,Apr 12 02:27PM,The Fillmore Silver Spring,2022-04-12,1,285
8,Apr 12 02:27PM,The Fillmore Silver Spring,2022-04-12,2,564
9,Apr 12 02:27PM,Brooklyn Bowl - Philadelphia,2022-04-13,2,147
